In [32]:
%run my_import

#uri = socket.gethostname()+".local:5555"
uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
#uri = "com-exp.local:5555"
#uri = "advantech.local:5555"
#uri = "192.168.1.100:5555"

io = set_uri(uri)
io.debug = False
scan_ids = list(reply_cmd(master_cmd_get_slave_descr).keys())
print('Found ESC ids {}'.format(scan_ids))

/home/amargan/work/anaconda3/envs/repl/bin/python
new uri localhost:5555
Found ESC ids [696]


In [11]:
iD = scan_ids[0]
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))
sdo_dict

{'adc_conv': 0.0,
 'adc_idx': 0,
 'adc_raw': 0,
 'analog_sample_freq_HZ': 20000,
 'aux_get': 31969.0,
 'aux_set': 31969.0,
 'board_id': 696,
 'control': 0.0,
 'ctrl_status_cmd': 0,
 'ctrl_status_cmd_ack': 0,
 'dac_offset': 0,
 'dac_tune_bitflag': 63,
 'dac_tune_hysteresis': 5,
 'fault_ack': 0,
 'flash_params_cmd': 0,
 'flash_params_cmd_ack': 0,
 'force_x': -1.595409,
 'force_y': -0.099651,
 'force_z': 6.978653,
 'fw_ver': 23022209,
 'module_params': 0,
 'offset': 0.0,
 'op_idx_ack': 257,
 'op_idx_aux': 257,
 'rtt': 31969.0,
 'smpl_num_pow2': 5,
 'status': 0.0,
 'temperature': 0.0,
 'torque_x': 0.539755,
 'torque_y': -7.543643,
 'torque_z': 1.277818,
 'ts': 31969.0}

In [12]:
flash_param = ['fw_ver','analog_sample_freq_HZ','dac_tune_bitflag','smpl_num_pow2','dac_tune_hysteresis']
sdo_filter(flash_param, sdo_dict)

{'fw_ver': 23022209,
 'analog_sample_freq_HZ': 20000,
 'dac_tune_bitflag': 63,
 'smpl_num_pow2': 5,
 'dac_tune_hysteresis': 5}

In [20]:
read_sdo(['smpl_num_pow2'],[iD])

{696: {'smpl_num_pow2': 5}}

In [13]:
write_sdo({'smpl_num_pow2': 1},[iD])

{696: {'smpl_num_pow2': 1}}

## Define funtions

In [14]:
def read_channel(bid:int, ch:int)->dict:
    write_sdo({'adc_idx':ch},[bid])
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

def read_channel_offset(bid:int, ch:int, offset:int) :
    write_sdo({'adc_idx': ch,'dac_offset':offset},[bid])
    reply_cmd(ctrl_cmd_set_dac.set_bid(bid))
    reply_cmd(ctrl_cmd_get_adc.set_bid(bid))
    return read_sdo(['adc_conv','adc_raw'],[bid])[bid]

def reset_DAC() :
    for ch in range(6) :
        ret = read_channel_offset(iD,ch,0x7FFF)
        print(ret)

def tune_adc_chan(bid:int, adc_idx:int) :
    dac_offset = 0x7FFF
    hister = 5
    tuned = False
    while ( not tuned ):
        yaml_msg = read_channel_offset(bid,adc_idx,dac_offset)
        adc_raw = yaml_msg['adc_raw']
        adc_conv = yaml_msg['adc_conv']
        if ( adc_raw > 0x1FFF+hister ) :
            if ( dac_offset < 0xFFFF ) :
                #dac_offset += 2
                dac_offset -= 2
        elif ( adc_raw < 0x1FFF-hister ) :
            if ( dac_offset > 0 ) :
                #dac_offset -= 2
                dac_offset += 2
        else : 
            tuned = True
        time.sleep(0.001)
        yield(tuned, adc_raw, adc_conv, dac_offset)

## Load default calib matrix

In [15]:
ctrl_status_cmd_str('LOAD_DFLT_CM', iD)

0x78c1


True

## read 1 channel

In [16]:
read_channel(iD,0)

{'adc_conv': 0.000915, 'adc_raw': 8197}

In [17]:
read_channel_offset(iD,0,0x7FFF)

{'adc_conv': 0.003965, 'adc_raw': 8217}

## read all 6 channels once

In [18]:

adc_conv = list()
adc_raw = list()
for j in range(1):
    for i in range(6):
        yaml_msg = read_channel(iD,i)
        adc_raw.append(yaml_msg['adc_raw'])
        adc_conv.append(yaml_msg['adc_conv'])
        print(yaml_msg)
        time.sleep(0.1)
        
[hex(x) for x in adc_raw]

{'adc_conv': 0.00427, 'adc_raw': 8219}
{'adc_conv': 0.00305, 'adc_raw': 8211}
{'adc_conv': -0.000915, 'adc_raw': 8185}
{'adc_conv': 0.009302, 'adc_raw': 8252}
{'adc_conv': -0.002135, 'adc_raw': 8177}
{'adc_conv': 0.00427, 'adc_raw': 8219}


['0x201b', '0x2013', '0x1ff9', '0x203c', '0x1ff1', '0x201b']

## reset DAC 

In [19]:
reset_DAC()

{'adc_conv': 0.00549, 'adc_raw': 8227}
{'adc_conv': 0.014335, 'adc_raw': 8285}
{'adc_conv': -0.000152, 'adc_raw': 8190}
{'adc_conv': 0.05307, 'adc_raw': 8539}
{'adc_conv': -0.016165, 'adc_raw': 8085}
{'adc_conv': 0.044682, 'adc_raw': 8484}


## Set all analog channel to Zero using DAC

In [20]:
for ch in range(6) :
    print("--- {0} ---".format(ch))
    for tuned, adc_raw, adc_conv, offset in tune_adc_chan(iD,ch) :
        print("\r%d %f 0x%X 0x%X"%(tuned, adc_conv, adc_raw, offset), end="")
        if tuned :
            write_sdo({'adc_idx': ch,'dac_offset':offset},[iD])
            reply_cmd(ctrl_cmd_set_dac_flash.set_bid(iD))
    print()


--- 0 ---
1 0.000610 0x2003 0x7FE5
--- 1 ---
1 -0.000305 0x1FFD 0x7F4D
--- 2 ---
1 -0.000610 0x1FFB 0x7FFF
--- 3 ---
1 0.000762 0x2004 0x7D5D
--- 4 ---
1 -0.000610 0x1FFB 0x80B1
--- 5 ---
1 -0.000305 0x1FFD 0x7DB3


In [23]:
write_sdo({'dac_tune_bitflag': 0x3F},[iD])
reply_cmd(flash_cmd_save_flash.set_bid(iD))

'Ok !!!'

In [22]:
write_sdo({'smpl_num_pow2': 5},[iD])

{696: {'smpl_num_pow2': 5}}

In [24]:
reply_cmd(ctrl_cmd_set_dac_flash.set_bid(iD))

'Ok !!!'

In [ ]:
cnt = 10
dac_offset = 0x7FFF
while (cnt) :
    cnt -= 1
    print(read_channel_offset(iD,0,dac_offset))
    dac_offset += 10


In [22]:
ctrl_status_cmd(0x0012, 696)

0x7812


True

In [23]:
ctrl_status_cmd(0x0021, 696)

0xaa21


False